In [35]:

!git clone https://github.com/sudeonder/wsi-retrieval.git
%cd wsi-retrieval

import sys
sys.path.append("wsi-retrieval")


Cloning into 'wsi-retrieval'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 74 (delta 23), reused 51 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 531.11 KiB | 4.05 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/wsi-retrieval


In [29]:
# Install required build tools and compile crcmod from source
!apt-get install -y python3-dev
!pip install --no-binary :all: crcmod

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp311-cp311-linux_x86_64.whl size=31654 sha256=fe445dfa0f8bbc02119be5265877b9de608ec4eb863082cc401a61590779960d
  Stored in directory: /root/.cache/pip/wheels/23/94/7a/8cb7d14597e6395ce969933f01aed9ea8fa5f5b4d4c8a61e99
Successfully built crcmod


In [6]:
# Install necessary packages
!apt-get install -y openslide-tools
!pip install openslide-python numpy pillow tqdm


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 104 kB of archives.
After this operation, 297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenslide0 amd64 3.4.1+dfsg-5build1 [89.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openslide-tools amd64 3.4.1+dfsg-5build1 [13.8 kB]
Fetched 104 kB in 0s (521 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-5build1_amd64.deb ...
Unpacking libopenslide0 (3.4.1+dfsg-5build1) ...
Selecting previously unselected package openslide-tools.

In [41]:
import os
from pathlib import Path
from utils.tile_utils import tile_wsi_if_occupied


In [12]:
# 📦 Step 1: Install dependencies
!pip install --quiet openslide-python
!apt-get install -y -qq openslide-tools
!pip install --upgrade google-cloud-storage

# 📂 Step 2: Set up GCS access
from google.colab import auth
auth.authenticate_user()

from google.cloud import storage
from pathlib import Path
import os



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-storage-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.90.0 requires google-cloud-storage<3.0.0,>=1.32.0, but you have google-cloud-storage 3.1.0 which is incompatible.


In [15]:
# ✅ Step 2: Authenticate & Mount Google Cloud Storage
from google.colab import auth
auth.authenticate_user()

# Set your bucket name here
BUCKET_NAME = "bracs-dataset-bucket"

# Mount Google Cloud Storage
from google.colab import drive
drive.mount('/content/drive')  # Optional if saving to Drive

from google.colab import auth
auth.authenticate_user()


Mounted at /content/drive


In [37]:
# ✅ Step 3: Import libraries and function
from pathlib import Path
import os
import subprocess
from utils.tile_utils import tile_wsi_if_occupied  # assumes you've written the .py file

# WSI paths
splits = ['train']  # Extend to ['train', 'val', 'test'] as needed
wsi_root_gcs = f"gs://{BUCKET_NAME}/BRACS/BRACS_WSI"
local_wsi_root = Path("/content/wsi")
output_tile_root = Path("/content/tiles")


In [38]:
# ✅ Step 4: Helper to list files in GCS folder using gsutil
def list_wsi_files(split: str):
    """
    Recursively list all .svs files in the split directory on GCS.
    Assumes WSIs are stored under split/Group_*/Type_*/WSI.svs.
    """
    wsi_root = f"{wsi_root_gcs}/{split}"
    print(f"🔍 Listing WSIs under: {wsi_root}/**/*.svs")

    # Recursive listing
    find_cmd = f"gsutil ls {wsi_root}/**.svs"
    result = subprocess.run(find_cmd.split(), capture_output=True, text=True)

    if result.returncode != 0:
        print("❌ Error while listing WSI files.")
        print(result.stderr)
        return []

    wsi_files = result.stdout.strip().split('\n')
    print(f"✅ Found {len(wsi_files)} WSIs.")
    return wsi_files


In [42]:
def tile_wsi_split_iteratively(
    split: str,
    max_wsi: int = None,
    patch_size: int = 448,
    stride: int = 448,
    patch_occupancy_threshold: float = 0.5,
    level: int = 0
):
    print(f"\n🔁 Tiling WSIs for split: {split}")

    wsi_files = list_wsi_files(split)
    if max_wsi:
        wsi_files = wsi_files[:max_wsi]

    print(f"✅ Found {len(wsi_files)} WSIs.\n")

    for i, wsi_gcs_path in enumerate(wsi_files, 1):
        rel_path = Path(wsi_gcs_path).relative_to(f"{wsi_root_gcs}/{split}")
        slide_id = rel_path.stem

        print(f"🔽 [{i}/{len(wsi_files)}] Downloading {slide_id}...")

        local_wsi_path = local_wsi_root / Path(wsi_gcs_path).relative_to(wsi_root_gcs)
        output_dir = output_tile_root / split / slide_id

        local_wsi_path.parent.mkdir(parents=True, exist_ok=True)
        output_dir.mkdir(parents=True, exist_ok=True)

        try:
            subprocess.run(["gsutil", "cp", wsi_gcs_path, str(local_wsi_path)], check=True)


            tile_wsi_if_occupied(
                wsi_path=local_wsi_path,
                output_dir=output_dir,
                patch_size=patch_size,
                stride=stride,
                level=level,
                patch_occupancy_threshold=patch_occupancy_threshold
            )

        except Exception as e:
            print(f"❌ Error processing {slide_id}: {e}")

        finally:
            if local_wsi_path.exists():
                local_wsi_path.unlink()
                print(f"🧹 Deleted {slide_id}.svs from local storage.")


In [43]:
tile_wsi_split_iteratively(
    split='train',
    max_wsi=2,  # limit to 2 WSIs for testing
    patch_size=448,
    stride=448
)



🔁 Tiling WSIs for split: train
🔍 Listing WSIs under: gs://bracs-dataset-bucket/BRACS/BRACS_WSI/train/**/*.svs
✅ Found 395 WSIs.
✅ Found 2 WSIs.

🔽 [1/2] Downloading BRACS_1003728...
📊 Slide occupancy for BRACS_1003728: 3.22%
⚠️ Skipping BRACS_1003728 due to low slide-level occupancy.
🧹 Deleted BRACS_1003728.svs from local storage.
🔽 [2/2] Downloading BRACS_1379...
📊 Slide occupancy for BRACS_1379: 4.80%
⚠️ Skipping BRACS_1379 due to low slide-level occupancy.
🧹 Deleted BRACS_1379.svs from local storage.


In [ ]:
!gsutil ls -r gs://bracs-dataset-bucket/BRACS/BRACS_WSI/train/**.svs